In [ ]:
#try using https://daler.github.io/pybedtools/autodocs/pybedtools.bedtool.BedTool.window_maker.html

In [87]:
import os
import argparse
import pandas as pd
from pybedtools import BedTool

In [79]:
parser = argparse.ArgumentParser(description='TFBS_coverage_rw')
parser.add_argument('directory_path', type=str, help='Location of base directory')
parser.add_argument('file_names', type=str, help='Name of folder and filenames for the promoters extracted')
parser.add_argument('promoter_bedfile', type=str, help='Input location of promoter bedfile')
parser.add_argument('motifs_bed', type=str, help='Input location of motifs bed file')
parser.add_argument('TFBS_coverage_bed', type=str, help='Output location of rolling window bed file')
parser.add_argument('window_bed', type=str, help='Output location of rolling window bed file')
parser.add_argument('window_size', type=int, help='Size of the rolling window in bp')
parser.add_argument('step_size', type=int, help='Size of the window offset in bp')


_StoreAction(option_strings=[], dest='window_gap', nargs=None, const=None, default=None, type=<class 'int'>, choices=None, help='Size of the gap between each rolling window in bp', metavar=None)

In [97]:
def window_split(promoter_bed, motifs_bed, output_bed, window_size, step_size):
    """function to split promoters into rolling windows"""
    proms = BedTool(promoter_bed)
    #make the sliding windows
    #w = window size
    #s = step size
    #n = no. of windows
    #i = srcwinnum (use source interval name with the window number)
    windows = BedTool().window_maker(b=proms, w=window_size, s=step_size, i='srcwinnum')
    
    with open(output_bed, 'w') as output:
        output.write(str(windows))
    return windows

In [84]:
def window_splitold(promoter_bed, motifs_bed, output_bed, window_size, step_size):
    """function to split promoters into rolling windows"""
    #read promoters and motifs into dfs and add column names
    proms = pd.read_table(promoter_bed, sep='\t')
    motifs = pd.read_table(motifs_bed, sep='\t')
    cols = ['chr', 'start', 'stop', 'AGI','dot1','strand','source','type','dot2','attributes']
    cols2 = ['chr','correct_start','correct_stop', 'motif_id', 'score', 'strand_x', 'sequence_name', 'p-value', 'q-value', 'matched_sequence']
    proms.columns = cols
    motifs.columns = cols2
    #rolling window

    return proms

In [91]:
proms = window_split(promoter_bedfile,motifs_bed,window_bed, window_size, step_size)

In [96]:
file_names = 'non-overlapping_includingbidirectional_all_genes_newannotationold'
directory_path = '../..'
promoter_bedfile = f'../../data/output/{file_names}/FIMO/promoters_5UTR.bed'
motifs_bed = f'../../data/output/{file_names}/FIMO/promoters_5UTR_motifs.bed'
TFBS_coverage_bed = f'{directory_path}/data/output/{file_names}/rolling_window/TFBS_coverage_rw/promoters_5UTR_TFBS_coverage_rw.bed'
window_bed = f'{directory_path}/data/output/{file_names}/rolling_window/promoters_5UTR_windows.bed'
window_size = 100
step_size = 50

In [12]:
#make directory for the output files to be exported to
#dirName = f'{args.directory_path}/data/output/{args.file_names}'
dirName = f'{directory_path}/data/output/{file_names}'
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " created") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  ../../data/output/non-overlapping_includingbidirectional_all_genes_newannotation  already exists


In [13]:
#make directory for the output files to be exported to
#dirName = f'{args.directory_path}/data/output/{args.file_names}'
dirName = f'{directory_path}/data/output/{file_names}/rolling_window'
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " created") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  ../../data/output/non-overlapping_includingbidirectional_all_genes_newannotation/rolling_window  created


In [14]:
#make directory for the output files to be exported to
#dirName = f'{args.directory_path}/data/output/{args.file_names}'
dirName = f'{directory_path}/data/output/{file_names}/rolling_window/TFBS_coverage_rw'
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " created") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  ../../data/output/non-overlapping_includingbidirectional_all_genes_newannotation/rolling_window/TFBS_coverage_rw  created
